In [1]:
from google.colab import drive
import os

from google.colab import drive
drive.mount('/content/drive')


# Corrected folder paths
dataset_path = '/content/drive/MyDrive/anomaly_detection_test_data'

good_images_path = os.path.join(dataset_path, 'good')
bad_images_path = os.path.join(dataset_path, 'bad')
masks_path = os.path.join(dataset_path, 'masks')

# Verify dataset structure again
print("Verifying dataset structure...")
print(f"Good images folder: {good_images_path}")
print(f"Bad images folder: {bad_images_path}")

# List files in good and bad image folders
print("\nSample 'good' images:")
print(os.listdir(good_images_path)[:5])

print("\nSample 'bad' images:")
print(os.listdir(bad_images_path)[:5])


Mounted at /content/drive
Verifying dataset structure...
Good images folder: /content/drive/MyDrive/anomaly_detection_test_data/good
Bad images folder: /content/drive/MyDrive/anomaly_detection_test_data/bad

Sample 'good' images:
['24_08_2024_18_12_36.034849_cls_input.png', '24_08_2024_18_07_39.184533_cls_input.png', '14_10_2024_13_04_36.123688_cls_input.png', '24_08_2024_18_09_22.348763_cls_input.png', '16_08_2024_12_37_56.690897_classifier_input.png']

Sample 'bad' images:
['03_08_2024_17_12_41.304965_classifier_input.png', 'Code03117.png', 'Code02553.png', '09_08_2024_18_36_59.620468_classifier_input.png', '16_08_2024_16_51_27.973379_classifier_input.png']


Data Preproccessing

In [2]:
from torchvision import transforms

# Define preprocessing transformations
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Testing the preprocessing on a sample image
from PIL import Image

# Load a sample image
sample_image_path = os.path.join(good_images_path, os.listdir(good_images_path)[0])
image = Image.open(sample_image_path).convert('RGB')

# Apply preprocessing
preprocessed_image = image_transforms(image)

print(f"Original Image Size: {image.size}")
print(f"Preprocessed Image Shape: {preprocessed_image.shape}")


Original Image Size: (333, 138)
Preprocessed Image Shape: torch.Size([3, 224, 224])


How to calculate Mean and STD fo this dataset

In [3]:

from torchvision import transforms
from PIL import Image
import os
import numpy as np
from tqdm import tqdm

# Function to calculate mean and std
def calculate_mean_std(image_paths):
    pixel_sum = np.zeros(3)
    pixel_squared_sum = np.zeros(3)
    num_pixels = 0

    for image_path in tqdm(image_paths, desc="Calculating mean and std"):
        try:
            # Open image and convert to RGB
            image = Image.open(image_path).convert('RGB')
            image = np.array(image) / 255.0  # Normalize to [0, 1]

            # Sum of pixels across channels
            pixel_sum += image.sum(axis=(0, 1))
            pixel_squared_sum += (image ** 2).sum(axis=(0, 1))

            # Total number of pixels
            num_pixels += image.shape[0] * image.shape[1]
        except Exception as e:
            print(f"Skipping file {image_path}: {e}")

    # Mean and std calculation
    mean = pixel_sum / num_pixels
    std = np.sqrt(pixel_squared_sum / num_pixels - mean ** 2)

    return mean, std

# Get all valid image paths
def get_valid_image_paths(folder_path, valid_extensions=('.jpg', '.jpeg', '.png')):
    return [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.lower().endswith(valid_extensions)]

# Get all image paths
good_image_paths = get_valid_image_paths(good_images_path)
bad_image_paths = get_valid_image_paths(bad_images_path)
all_image_paths = good_image_paths + bad_image_paths

# Calculate mean and std
mean, std = calculate_mean_std(all_image_paths)

print(f"Calculated Mean: {mean}")
print(f"Calculated Std: {std}")


Calculating mean and std: 100%|██████████| 5180/5180 [09:00<00:00,  9.59it/s]

Calculated Mean: [0.7342358 0.7342358 0.7342358]
Calculated Std: [0.33371068 0.33371068 0.33371068]


Code For Preprocessing

In [4]:
from torchvision import transforms

# Define preprocessing transformations
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert to PyTorch tensor
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    transforms.Normalize(mean=mean, std=std)  # Normalize to [-1, 1]
])

# Testing the preprocessing on a sample image
from PIL import Image

# Load a sample image
sample_image_path = os.path.join(good_images_path, os.listdir(good_images_path)[0])
image = Image.open(sample_image_path).convert('RGB')

# Apply preprocessing
preprocessed_image = image_transforms(image)

print(f"Original Image Size: {image.size}")
print(f"Preprocessed Image Shape: {preprocessed_image.shape}")


Original Image Size: (333, 138)
Preprocessed Image Shape: torch.Size([3, 224, 224])


Custom dataset class

In [5]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ImageClassificationDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        """
        Args:
            image_paths (list): List of image file paths.
            labels (list): Corresponding labels for each image.
            transform (callable, optional): Transformation to apply to the images.
        """
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load the image
        image = Image.open(self.image_paths[idx]).convert('RGB')

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)

        # Get the label
        label = self.labels[idx]

        return image, label


Dataset spilitng and Dataset Loader Preparation

In [6]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
from torchvision import transforms

# Define transformations
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert to PyTorch tensor
    transforms.Normalize(mean=mean, std=std) # Normalize to [-1, 1]
])

# Prepare image paths and labels
good_image_paths = get_valid_image_paths(good_images_path)
bad_image_paths = get_valid_image_paths(bad_images_path)

all_image_paths = good_image_paths + bad_image_paths
all_labels = [0] * len(good_image_paths) + [1] * len(bad_image_paths)  # 0 for good, 1 for bad

from sklearn.model_selection import train_test_split

# Split dataset into train (70%) and temp (30%)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    all_image_paths, all_labels, test_size=0.3, random_state=42, stratify=all_labels
)

# Split temp (30%) into validation (15%) and test (15%)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Print sizes to verify
print(f"Training set size: {len(train_paths)}")
print(f"Validation set size: {len(val_paths)}")
print(f"Testing set size: {len(test_paths)}")

# Create datasets
train_dataset = ImageClassificationDataset(train_paths, train_labels, transform=image_transforms)
val_dataset = ImageClassificationDataset(val_paths, val_labels, transform=image_transforms)
test_dataset = ImageClassificationDataset(test_paths, test_labels, transform=image_transforms)

# Create DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Test the DataLoaders
for images, labels in train_loader:
    print(f"Batch of images shape: {images.shape}")
    print(f"Batch of labels shape: {labels.shape}")
    break


Training set size: 3626
Validation set size: 777
Testing set size: 777
Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels shape: torch.Size([32])


Model Setup

In [7]:
import torch
import torch.nn as nn
from torchvision import models

# Load pre-trained ResNet18
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer for binary classification
num_features = model.fc.in_features  # Get the number of input features for the final FC layer
model.fc = nn.Linear(num_features, 2)  # Replace with a new layer with 2 output classes (good/bad)

# Move the model to the correct device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Print the modified model
print(model)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 114MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Code For training setup

In [8]:
import torch
import torch.nn as nn
from torch.optim import Adam

# Define loss function
loss_function = nn.CrossEntropyLoss()

# Define optimizer
learning_rate = 1e-4
optimizer = Adam(model.parameters(), lr=learning_rate)

# Number of epochs
num_epochs = 10

# Function to calculate accuracy
def calculate_accuracy(predictions, labels):
    _, predicted_classes = torch.max(predictions, 1)
    correct = (predicted_classes == labels).sum().item()
    accuracy = correct / len(labels)
    return accuracy


Code For Training Loop

In [9]:

from tqdm import tqdm
import torch

# Function to calculate accuracy
def calculate_accuracy(predictions, labels):
    _, predicted_classes = torch.max(predictions, 1)
    correct = (predicted_classes == labels).sum().item()
    accuracy = correct / len(labels)
    return accuracy

# Training loop
best_val_accuracy = 0.0  # Track the best validation accuracy
save_path = "best_model.pth"  # Specify where to save the model

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    # Training phase
    model.train()  # Set model to training mode
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = loss_function(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate training loss
        train_loss += loss.item()

        # Calculate training accuracy
        _, predicted_classes = torch.max(outputs, 1)
        train_correct += (predicted_classes == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    avg_train_loss = train_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # Disable gradient computation
        for images, labels in tqdm(val_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_function(outputs, labels)

            # Accumulate validation loss
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted_classes = torch.max(outputs, 1)
            val_correct += (predicted_classes == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Save the best model based on validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), save_path)
        print(f"Best model saved with Validation Accuracy: {best_val_accuracy:.4f}")




Epoch 1/10


Training: 100%|██████████| 114/114 [00:22<00:00,  4.98it/s]


Training Loss: 0.1589, Training Accuracy: 0.9418


Validation: 100%|██████████| 25/25 [00:03<00:00,  6.38it/s]


Validation Loss: 0.0365, Validation Accuracy: 0.9910
Best model saved with Validation Accuracy: 0.9910

Epoch 2/10


...

Validation Loss: 0.0278, Validation Accuracy: 0.9910

Epoch 10/10


Training: 100%|██████████| 114/114 [00:21<00:00,  5.32it/s]


Training Loss: 0.0010, Training Accuracy: 1.0000


Validation: 100%|██████████| 25/25 [00:05<00:00,  4.18it/s]

Validation Loss: 0.0204, Validation Accuracy: 0.9949


Testing the model

In [10]:
# Testing the model
def test_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient computation
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_function(outputs, labels)
            test_loss += loss.item()

            # Calculate accuracy
            _, predicted_classes = torch.max(outputs, 1)
            correct_predictions += (predicted_classes == labels).sum().item()
            total_samples += labels.size(0)

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = correct_predictions / total_samples
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Run testing
test_model(model, test_loader)


Testing: 100%|██████████| 25/25 [00:04<00:00,  6.10it/s]

Test Loss: 0.0372, Test Accuracy: 0.9910


Saving the testing model

In [11]:
# Save the trained model
torch.save(model.state_dict(), "resnet18_good_bad_classification.pth")
print("Model saved successfully!")


Model saved successfully!


Load and use the trained model

In [14]:
# Load the saved model
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(num_features, 2)  # Ensure the same structure as trained model
model.load_state_dict(torch.load("resnet18_good_bad_classification.pth"))
model = model.to(device)
model.eval()  # Set to evaluation mode

# Predict on a new image
from PIL import Image
new_image_path = "/content/drive/MyDrive/anomaly_detection_test_data/bad/Code03775.png"  # Replace with the actual path
image = Image.open(new_image_path).convert('RGB')

# Preprocess the image
preprocessed_image = image_transforms(image).unsqueeze(0).to(device)  # Add batch dimension

# Perform prediction
with torch.no_grad():
    output = model(preprocessed_image)
    _, predicted_class = torch.max(output, 1)
    print(f"Predicted Class: {'Good' if predicted_class.item() == 0 else 'Bad'}")


Predicted Class: Bad


<ipython-input-14-b5fcc4357b73>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("resnet18_good_bad_classification.pth"))
